In [1]:
import cv2
import numpy as np
import mediapipe as mp
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

DRAW_COLOR = (255, 255, 255)  
ERASE_COLOR = (0, 0, 0)  
DRAW_THICKNESS = 10
ERASER_THICKNESS = 40
LANDMARK_COLOR = (0, 255, 255)  
LINE_COLOR = (0, 255, 0)  
LINE_THICKNESS = 3 

ICONS = {
    'blue': (100, 50, (255, 0, 0)),  
    'green': (200, 50, (0, 255, 0)), 
    'eraser': (300, 50, (255, 255, 255)),  
    'predict': (400, 50, (255, 255, 255))  
}
ICON_RADIUS = 30
ICON_HIGHLIGHT_SCALE = 2  


model = load_model('mnist_cnn_model2.h5')


HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),  
    (0, 5), (5, 6), (6, 7), (7, 8),  
    (0, 9), (9, 10), (10, 11), (11, 12),  
    (0, 13), (13, 14), (14, 15), (15, 16),  
    (0, 17), (17, 18), (18, 19), (19, 20)  
]

def is_clicked(landmarks, index1, index2, threshold=30):
    x1, y1 = int(landmarks[index1].x * width), int(landmarks[index1].y * height)
    x2, y2 = int(landmarks[index2].x * width), int(landmarks[index2].y * height)
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2) < threshold

def draw_icons(frame, highlighted_icon=None):
    for icon, (x, y, color) in ICONS.items():
        radius = ICON_RADIUS * ICON_HIGHLIGHT_SCALE if highlighted_icon == icon else ICON_RADIUS
        cv2.circle(frame, (x, y), radius, color, -1)
        if icon == 'predict':
            cv2.putText(frame, 'p', (x - 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 1 * ICON_HIGHLIGHT_SCALE, (0, 0, 0), 2)

def preprocess_for_model(image):
    if len(image.shape) == 3 and image.shape[2] == 3:  
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    else:
        gray_image = image  

    resized_image = cv2.resize(gray_image, (28, 28), interpolation = cv2.INTER_AREA)
    normailzed_image = tf.keras.utils.normalize(resized_image, axis = 1)
    input_data = np.array(normailzed_image).reshape(-1,28,28,1)  
    return input_data

def draw_landmarks_and_connections(frame, landmarks):
    for lm in landmarks:
        x, y = int(lm.x * width), int(lm.y * height)
        cv2.circle(frame, (x, y), 5, LANDMARK_COLOR, -1)  

    for connection in HAND_CONNECTIONS:
        start_idx, end_idx = connection
        start_point = (int(landmarks[start_idx].x * width), int(landmarks[start_idx].y * height))
        end_point = (int(landmarks[end_idx].x * width), int(landmarks[end_idx].y * height))
        cv2.line(frame, start_point, end_point, LINE_COLOR, LINE_THICKNESS)  

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

canvas = np.zeros((480, 640, 3), dtype="uint8")  
color_canvas = np.zeros((480, 640, 3), dtype="uint8")  
prediction_canvas = np.zeros_like(canvas)  
current_color = None  
drawing = False
last_point = None
highlighted_icon = None
predicted_digit = None  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        landmarks = result.multi_hand_landmarks[0].landmark

        
        draw_landmarks_and_connections(frame, landmarks)

        thumb_tip = (int(landmarks[4].x * width), int(landmarks[4].y * height))
        index_tip = (int(landmarks[8].x * width), int(landmarks[8].y * height))

        
        if is_clicked(landmarks, 4, 8, threshold=30):
            new_highlighted_icon = None
            for icon, (x, y, color) in ICONS.items():
                if np.sqrt((index_tip[0] - x)**2 + (index_tip[1] - y)**2) < 30:
                    new_highlighted_icon = icon
                    if icon == 'blue':
                        current_color = (255, 0, 0)
                    elif icon == 'green':
                        current_color = (0, 255, 0)
                    elif icon == 'eraser':
                        current_color = ERASE_COLOR
                    elif icon == 'predict':
                        
                        gray_canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
                        input_data = preprocess_for_model(gray_canvas)
                        
                        prediction = model.predict(input_data).argmax()
                        predicted_digit = prediction
                        print(f"Predicted Digit: {predicted_digit}")  

            highlighted_icon = new_highlighted_icon

            
            if current_color:
                if drawing:
                    drawing = False
                    last_point = None
                else:
                    drawing = True

        if drawing and result.multi_hand_landmarks:
            index_tip = (int(landmarks[8].x * width), int(landmarks[8].y * height))
            if last_point:
                if current_color == ERASE_COLOR:
                    cv2.line(canvas, last_point, index_tip, ERASE_COLOR, ERASER_THICKNESS)
                    cv2.line(color_canvas, last_point, index_tip, ERASE_COLOR, ERASER_THICKNESS)  
                else:
                    cv2.line(canvas, last_point, index_tip, DRAW_COLOR, DRAW_THICKNESS)  
                    cv2.line(color_canvas, last_point, index_tip, current_color, DRAW_THICKNESS)  
                    cv2.line(prediction_canvas, last_point, index_tip, DRAW_COLOR, DRAW_THICKNESS)  
            last_point = index_tip

    
    draw_icons(frame, highlighted_icon)

    
    overlay = cv2.addWeighted(frame, 0.7, color_canvas, 0.3, 0)

    
    if predicted_digit is not None:
        box_x1, box_y1 = width - 150, height - 100  
        box_x2, box_y2 = width - 10, height - 20  
        cv2.rectangle(overlay, (box_x1, box_y1), (box_x2, box_y2), (0, 0, 0), -1)
        cv2.putText(overlay, f"{predicted_digit}", (box_x1 + 20, box_y2 - 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    cv2.imshow("Webcam Feed", overlay)
    

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\rohan\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted Digit: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Digit: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Digit: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted Digit: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Digit: 7
